In [16]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras import callbacks as call
#from sklearn.preprocessing import LabelEncoder
#from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.layers.core import Reshape

from tensorflow import keras
layers = keras.layers

In [17]:
print("Running TensorFlow Version number: ", tf.__version__)

Running TensorFlow Version number:  2.1.0


In [18]:
data = pd.read_csv("./owid-covid-data.csv")

#print(data)

In [44]:
trainSize = int(len(data) * 0.8)     #80% data set for training

#Training features
trainData = data[1:trainSize][:]
trainData.pop('total_deaths')         # The label for the model

trainData.pop('iso_code')             # The values are removed from the data
trainData.pop('continent')            # because our data should not be biased based on location
trainData.pop('location')             # or by date, only by medical statistics
trainData.pop('date')
trainData.pop('tests_units')
trainData = tf.convert_to_tensor(np.asarray(trainData).astype(np.float32), dtype=tf.float32)

#Training labels
trainLabels = data[1:trainSize]['total_deaths']
trainLabels = tf.convert_to_tensor(np.asarray(trainLabels).astype(np.float32), dtype=tf.float32)

#Testing Features
testData = data[1+trainSize:][:]
testData.pop('total_deaths')          # The label for the model

testData.pop('iso_code')             # The values are removed from the data
testData.pop('continent')            # because our data should not be biased based on location
testData.pop('location')             # or by date, only by medical statistics
testData.pop('date')
testData.pop('tests_units')
testData = tf.convert_to_tensor(np.asarray(testData).astype(np.float32), dtype=tf.float32)

#Testing labels
testLabels = data[1+trainSize:]['total_deaths']
testLabels = tf.convert_to_tensor(np.asarray(testLabels).astype(np.float32), dtype=tf.float32)

#trainData   = trainData.numpy()
#trainLabels = trainLabels.numpy()
#testData    = testData.numpy()
#testLabels  = testLabels.numpy()

In [60]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Preprocess the data (these are NumPy arrays)
x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Reserve 10,000 samples for validation
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

11493376/11490434 [==============================] - 2s 0us/step


In [63]:
print(x_train.shape)

(50000, 784)


In [57]:
#print(trainData)
#print(type(trainData))
print(len(trainData))
print(len(trainData[0]))
#print(trainData.shape)
#print(tf.reshape(trainData, shape=(1,trainData.shape)))
#print(len(trainLabels))
#print(len(trainLabels))

26066
28
26066


TypeError: Scalar tensor has no `len()`

In [47]:
model = keras.Sequential() # Create the model

#x = Reshape((len(trainData),len(trainData[0])))(trainData)
x=(len(trainData),len(trainData[0]))
model.add(Conv1D(input_shape=x,filters=64,kernel_size=3,padding="same", activation="relu"))
model.add(Conv1D(filters=64,kernel_size=3,padding="same", activation="relu"))
model.add(MaxPooling1D(pool_size=2,strides=None))
# First 2 convolution layers with a maxpool of 2x2 and stride of 2x2

model.add(Conv1D(filters=128, kernel_size=3, padding="same", activation="relu"))
model.add(Conv1D(filters=128, kernel_size=3, padding="same", activation="relu"))
model.add(MaxPooling1D(pool_size=2,strides=None))
# second 2 convoltion layers with a maxpool of 2x2 and stride of 2x2

model.add(Conv1D(filters=256, kernel_size=3, padding="same", activation="relu"))
model.add(Conv1D(filters=256, kernel_size=3, padding="same", activation="relu"))
model.add(Conv1D(filters=256, kernel_size=3, padding="same", activation="relu"))
model.add(MaxPooling1D(pool_size=2,strides=None))
# third layer of 3 convolution layers with a maxpool of 2x2 and stride of 2x2

model.add(Conv1D(filters=512, kernel_size=3, padding="same", activation="relu"))
model.add(Conv1D(filters=512, kernel_size=3, padding="same", activation="relu"))
model.add(Conv1D(filters=512, kernel_size=3, padding="same", activation="relu"))
model.add(MaxPooling1D(pool_size=2,strides=None))
# fourth layer of 3 convolution layers with a maxpool of 2x2 and stride of 2x2

model.add(Conv1D(filters=512, kernel_size=3, padding="same", activation="relu"))
model.add(Conv1D(filters=512, kernel_size=3, padding="same", activation="relu"))
model.add(Conv1D(filters=512, kernel_size=3, padding="same", activation="relu"))
model.add(MaxPooling1D(pool_size=2,strides=None))
# fifth layer of 3 convolution layers with a maxpool of 2x2 and stride of 2x2

#Structure detailed by Karen Simonyan and Andrew Zisserman in
#VERY DEEP CONVOLUTIONAL NETWORKS FOR LARGE-SCALE IMAGE RECOGNITION
# But modified with a reduced dimenion for 2 dimensional data and not images


In [48]:
#model.add(Dropout(0.2))
#model.add(Flatten())

model.add(Dense(units=512,activation="relu"))
# First fully connected newtork of 4096 units

model.add(Dense(units=512,activation="relu"))
# Second fully connected newtork of 4096 units

model.add(Dense(units=256,activation="relu"))
# Third fully connected newtork of 1000 units

#model.add(Flatten())

model.add(Dense(units=1, activation="sigmoid"))
# The soft-max output


In [49]:

#opt = Adam(lr=0.01)

#model.compile(optimizer=opt, loss=tensorflow.keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])

#model.compile(loss='binary_crossentropy', 
#              optimizer='rmsprop', 
#              metrics=['accuracy']) 
sgd = tf.keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd, metrics=['accuracy'])


In [50]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_39 (Conv1D)           (None, 26066, 64)         5440      
_________________________________________________________________
conv1d_40 (Conv1D)           (None, 26066, 64)         12352     
_________________________________________________________________
max_pooling1d_15 (MaxPooling (None, 13033, 64)         0         
_________________________________________________________________
conv1d_41 (Conv1D)           (None, 13033, 128)        24704     
_________________________________________________________________
conv1d_42 (Conv1D)           (None, 13033, 128)        49280     
_________________________________________________________________
max_pooling1d_16 (MaxPooling (None, 6516, 128)         0         
_________________________________________________________________
conv1d_43 (Conv1D)           (None, 6516, 256)        

In [51]:
print(type(trainData))

<class 'tensorflow.python.framework.ops.EagerTensor'>


In [59]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

EPOCHS = 1000
training_samples = 4000
validation_samples = 600

snapshot_name = "Covid-19_snapshot_"
checkpoint = ModelCheckpoint(snapshot_name+".{epoch:02d}-{val_loss:.2f}.hdf5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

reduceLoss = EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=1, mode='auto')
'''
X_new = InputLayer(input_shape=(None, len(trainData),len(trainData[0])), input_tensor=trainData)
Y_new = InputLayer(input_shape=(None, len(trainLabels),len(trainLabels[0])), input_tensor=trainLabels)

history = model.fit(
                    X_new,Y_new, epochs=EPOCHS, validation_split=0.2, verbose=0,
                    callbacks=[checkpoint,reduceLoss]
                    )
'''
history = model.fit(
                    (len(trainData),len(trainData[0])), (len(trainLabels),1), epochs=EPOCHS, validation_split=0.2, verbose=0,
                    callbacks=[checkpoint,reduceLoss]
                    )

train_generator = generator(trainData, batch_size = 32)
validation_generator = generator(testData, batch_size = 32)

'''history = model.fit_generator(train_generator, 
                           steps_per_epoch=training_samples,
                           validation_data=validation_generator, 
                           validation_steps=validation_samples,
                           epochs=EPOCHS,
                           callbacks=[checkpoint,reduceLoss]
                          )
'''

ValueError: Failed to find data adapter that can handle input: (<class 'tuple'> containing values of types {"<class 'int'>", "<class 'NoneType'>"}), (<class 'tuple'> containing values of types {"<class 'int'>", "<class 'NoneType'>"})